In [ ]:
from __future__ import annotations
import datetime as dt
from typing import NamedTuple
from collections.abc import Iterable
from random import random
import sys
from contextlib import contextmanager

from tuplesaver.persister import Engine
from tuplesaver.adaptconvert import AdaptConvertTypeAlreadyRegistered, register_standard_adaptconverters
register_standard_adaptconverters()

In [ ]:
class ListType(NamedTuple):
    id: int | None
    codename: str


class List(NamedTuple):
    id: int | None
    name: str  # unique
    listtype: ListType

class ListSection(NamedTuple):
    id: int | None
    title: str
    list: List


class ListItem(NamedTuple):
    id: int | None
    txt: str
    listsection: ListSection


@contextmanager
def engine():
    engine = Engine("xxx.db")

    engine.ensure_table_created(ListType)
    engine.ensure_table_created(List)
    engine.ensure_table_created(ListSection)
    engine.ensure_table_created(ListItem)

    engine.connection.set_trace_callback(print)  # for debugging
    yield engine
    engine.connection.commit()
    engine.connection.close()


with engine() as e:
    shopping = e.save(ListType(None, "shopping"))
    groceries = e.save(List(None, "Grocery", shopping))


# POC of lazy loading relationships

In [ ]:
# query stub
from typing import cast

# monkey‑patch List so any LazyProxy field is transparently unwrapped
def _unwrap_lazyproxy_getattr(self, attr):
    value = object.__getattribute__(self, attr)
    if isinstance(value, Lazy):
        return value._obj()          # materialise & return real row
    return value

List.__getattribute__ = _unwrap_lazyproxy_getattr


def getrow[R: Row](Model: type[R], id_: int | None) -> R | None:
    if id_ is None:
        return None  # no row to look up
    # replace with real lookup
    with engine() as e:
        row = e.find(Model, id_)
    if row is None:
        raise KeyError(f"{Model.__name__}:{id_}")
    else:
        return row


from tuplesaver.model import Row

class Lazy[Model]():
    __slots__ = ("_model", "_id", "_cached")
    def __init__(self, model: type[Row], id_: int):
        self._model = model
        self._id = id_
        self._cached = None
    def _obj(self)-> Model:
        if self._cached is None:
            self._cached = getrow(self._model, self._id)
        return cast(Model, self._cached)
    def __repr__(self):
        return (f"<{self.__class__.__name__}[{self._model.__name__}]:{self._id}>"
                if self._cached is None else repr(self._cached))

# demo instance
lst_lazy = List(id=42, name="dummy_list", listtype=cast(ListType, Lazy(ListType,1)))
display(lst_lazy)
display(lst_lazy)
stolen = lst_lazy.listtype
print(stolen)
display(lst_lazy)

# Example of verbose loading because of lack of backrefs/lazy loading
use this to drive the design

In [ ]:
# populate the database with some data
with engine() as e:
    section_items = {
        'produce': [
            'Other fruit',
            'Berries',
            'Limes',
            'oranges, not the very large ones, but not the small ones either. Also they should be extra sweet, and heavy (and therefore JUICY 💦 yum!)',
        ],
        'deli': ['sliced cheese', 'Lunch meat', 'Goat cheese'],
        'meat': ['chicken'],
        'dairy': ['Milk', 'Eggs', 'Sour cream', 'Cheddar block'],
        'frozen': ['Frozen Veggies'],
        'drinks': ['sparkling water', 'Pop'],
        'snacks': ['peanuts', 'Triscut', 'Goldfish'],
        'ingredient': ['tortillas', 'Peanut butter', 'Prescription'],
        'etc': ['cat food', 'Tide'],
    }

    for section, items in section_items.items():
        section = e.save(ListSection(None, section, groceries))
        for item in items:
            e.save(ListItem(None, item, section))


In [ ]:
from tuplesaver.query import select

@select(ListItem)
def listitems_by_listname(list_name: str) -> str:
    return f"WHERE {ListItem.listsection.list.name} = {list_name}"


list_name = "Grocery"

@select(List)
def list_by_name(list_name: str) -> str:
    return f"WHERE {List.name} = {list_name}"
mylist = e.query(*list_by_name(list_name)).fetchone()

mylist  = e.find_by(List, name = list_name)
mylist  = List.find_by(name = list_name) # type: ignore[reportCallIssue]

with engine() as e:
    mylist  = e.find_by(List, name = list_name)


if mylist is None:
    print(f"List {list_name} not found")

with engine() as e:
    listitems = e.query(*listitems_by_listname(list_name)).fetchall()
if len(listitems) == 0:
    print(f"No items found for list {list_name}")

section_items = {}
for item in listitems:
    section_items.setdefault(item.listsection, []).append(item)

print(f"Rendering template for list {mylist}")


In [ ]:
# example of types of dataclasses vs dataclass instances

from typing import NamedTuple
from dataclasses import dataclass

class MyNamedTuple(NamedTuple):
    x: int
    y: str

@dataclass
class MyDataClass:
    x: int
    y: str

mnt = MyNamedTuple(1, "a")
mdc = MyDataClass(1, "a")

from typing import reveal_type
reveal_type(MyNamedTuple)  # N: Type[MyNamedTuple]
reveal_type(MyNamedTuple.x)  # N: int
reveal_type(mnt)           # N: MyNamedTuple
reveal_type(mnt.x)        # N: int

reveal_type(MyDataClass)   # N: Type[MyDataClass]
reveal_type(MyDataClass.x)  # N: error
reveal_type(mdc)          # N: MyDataClass
reveal_type(mdc.x)       # N: int


In [ ]:
MyDataClass.x